In [143]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score,RandomizedSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle


In [144]:
dataset = pd.read_csv('/content/autism.csv')

In [145]:
dataset.head()

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,1,0,1,0,1,0,1,...,f,?,no,no,Austria,no,6.351166,18 and more,Self,0
1,2,0,0,0,0,0,0,0,0,0,...,m,?,no,no,India,no,2.255185,18 and more,Self,0
2,3,1,1,1,1,1,1,1,1,1,...,m,White-European,no,yes,United States,no,14.851484,18 and more,Self,1
3,4,0,0,0,0,0,0,0,0,0,...,f,?,no,no,United States,no,2.276617,18 and more,Self,0
4,5,0,0,0,0,0,0,0,0,0,...,m,?,no,no,South Africa,no,-4.777286,18 and more,Self,0


In [146]:
dataset.columns

Index(['ID', 'A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age',
       'gender', 'ethnicity', 'jaundice', 'austim', 'contry_of_res',
       'used_app_before', 'result', 'age_desc', 'relation', 'Class/ASD'],
      dtype='object')

In [147]:
dataset = dataset.drop(['ID'], axis=1)

In [148]:
dataset.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'ethnicity', 'jaundice', 'austim', 'contry_of_res', 'used_app_before',
       'result', 'age_desc', 'relation', 'Class/ASD'],
      dtype='object')

In [149]:
dataset = dataset.replace({
    'gender': {'f': 0, 'm': 1},
    'ethnicity': {
        'White-European': 0, '?': 1, 'Middle Eastern ': 2, 'Asian': 3, 'Black': 4,
        'South Asian': 5, 'Pasifika': 6, 'Others': 7, 'Latino': 8,
        'Hispanic': 9, 'Turkish': 10, 'others': 11
    },
    'jaundice':{'yes':1,'no':0},
    'austim': {'yes': 1, 'no': 0},
    'used_app_before': {'yes': 1, 'no': 0},
    'relation': {'Self': 0, '?': 1,'Parent':2,'Relative':3,'Others':4,'Health care professional':5}
})


/tmp/ipython-input-1436654352.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.replace({


In [150]:
le = LabelEncoder()
dataset['contry_of_res'] = le.fit_transform(dataset['contry_of_res'])

In [151]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dataset['result'] = scaler.fit_transform(dataset[['result']])

In [152]:
dataset = dataset.drop(columns='age_desc',axis=1)

In [153]:
dataset.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'ethnicity', 'jaundice', 'austim', 'contry_of_res', 'used_app_before',
       'result', 'relation', 'Class/ASD'],
      dtype='object')

In [154]:
dataset['relation'].value_counts()

,count
relation,
0,709
1,40
2,29
3,18
4,2
5,2


In [155]:
dataset.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,0,1,0,1,0,1,0,1,1,38.172746,0,1,0,0,7,0,-0.455003,0,0
1,0,0,0,0,0,0,0,0,0,0,47.750517,1,1,0,0,25,0,-1.307503,0,0
2,1,1,1,1,1,1,1,1,1,1,7.380373,1,0,0,1,54,0,1.314176,0,1
3,0,0,0,0,0,0,0,0,0,0,23.561927,0,1,0,0,54,0,-1.303042,0,0
4,0,0,0,0,0,0,0,0,0,0,43.205790,1,1,0,0,46,0,-2.771177,0,0


In [156]:
xx = dataset.drop('Class/ASD',axis=1)
yy = dataset['Class/ASD']

In [157]:
sm = SMOTE(random_state=42)
x,y= sm.fit_resample(xx,yy)


In [158]:
x_test,x_train,y_test,y_train = train_test_split(x,y,test_size=0.2,random_state=42)

In [159]:
model1 = XGBClassifier()
model2 = RandomForestClassifier()
model3 = DecisionTreeClassifier()

In [160]:
model1.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [161]:
model2.fit(x_train,y_train)

RandomForestClassifier()

In [162]:
model3.fit(x_train,y_train)

DecisionTreeClassifier()

In [163]:
prediction = model1.predict(x_test)
acc = accuracy_score(y_test,prediction)
print(acc)

0.8639921722113503


In [164]:
prediction2 = model2.predict(x_test)
acc2 = accuracy_score(y_test,prediction2)
print(acc2)

0.8620352250489237


In [165]:
prediction3 = model3.predict(x_test)
acc3 = accuracy_score(y_test,prediction3)
print(acc3)

0.8052837573385518


In [166]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model2, file)

In [167]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [168]:
pred = loaded_model.predict(x_test.iloc[[1]])
if pred[0] == 0:
  print('No')
else:
  print('Yes')

No


In [169]:
x_test.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,relation
852,1,1,1,0,1,1,1,1,1,1,27.623227,0,0,0,0,54,0,0.922437,0
727,1,0,0,0,0,0,0,1,1,0,22.869820,1,7,0,0,13,0,0.128320,0
158,1,0,0,0,1,0,0,0,1,0,31.619380,1,8,0,0,20,0,-0.118800,2
351,1,1,1,0,1,1,0,0,0,1,17.116422,0,1,0,0,25,1,-0.709012,0
1108,1,1,1,1,1,1,1,1,1,1,29.019418,0,0,0,0,53,0,0.709662,0
